<a href="https://colab.research.google.com/github/jalevano/tfm_uoc_datascience/blob/main/03_Analisis_Fase_2C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
================================================================================
FASE 2C - PCA, CLUSTERING Y ANALISIS DE REDUNDANCIA
================================================================================
Trabajo Fin de Master: Evaluacion Comparativa de Tecnicas de Segmentacion
en Fotografia de Retrato

Autor: Jesus L.
Universidad: Universitat Oberta de Catalunya (UOC)
Fecha: Diciembre 2025

DESCRIPCION:
Fase 2C del analisis comparativo. Realiza analisis exploratorio multivariante
sobre TODAS las metricas de segmentacion disponibles.

VERSION 3.0 - CAMBIOS:
- Excluye boundary_iou (valores incorrectos por implementacion Canny)
- Usa chamfer_distance y hausdorff_distance como metricas de calidad de borde
- PCA global y por categoria de metricas
- Clustering de fotografias por dificultad
- Analisis de redundancia entre metricas

METRICAS UTILIZADAS (64 metricas):
- Clasicas: IoU, Dice, Precision, Recall, F1 (5)
- Distancias de borde: Chamfer, Hausdorff (2) - REEMPLAZAN boundary_iou
- Geometricas Shapely: 28 metricas
- Haralick interior: 13 metricas texturales
- Haralick borde: 13 metricas texturales
- Intensidad: 4 metricas

ESTRUCTURA DE SALIDA:
/TFM/3_Analisis/fase2c_pca_clustering/
├── pca_global/
│   ├── pca_scores.csv
│   ├── pca_loadings.csv
│   └── pca_varianza.csv
├── pca_por_categoria/
│   └── (CSV por categoria)
├── clustering/
│   ├── clusters_fotografias.csv
│   └── caracterizacion_clusters.json
├── correlaciones/
│   ├── matriz_correlaciones.csv
│   ├── correlaciones_con_iou.csv
│   └── metricas_redundantes.csv
├── visualizaciones/
│   └── (PNG generados)
├── resumen_fase2c.json

================================================================================
"""

'\n================================================================================\nFASE 2C - PCA, CLUSTERING Y ANALISIS DE REDUNDANCIA (VERSION 3.0)\n================================================================================\nTrabajo Fin de Master: Evaluacion Comparativa de Tecnicas de Segmentacion\nen Fotografia de Retrato\n\nAutor: Jesus L.\nUniversidad: Universitat Oberta de Catalunya (UOC)\nFecha: Diciembre 2025\n\nDESCRIPCION:\nFase 2C del analisis comparativo. Realiza analisis exploratorio multivariante\nsobre TODAS las metricas de segmentacion disponibles.\n\nVERSION 3.0 - CAMBIOS:\n- Excluye boundary_iou (valores incorrectos por implementacion Canny)\n- Usa chamfer_distance y hausdorff_distance como metricas de calidad de borde\n- PCA global y por categoria de metricas\n- Clustering de fotografias por dificultad\n- Analisis de redundancia entre metricas\n\nMETRICAS UTILIZADAS (64 metricas):\n- Clasicas: IoU, Dice, Precision, Recall, F1 (5)\n- Distancias de borde: Chamfe

In [3]:
# =============================================================================
# IMPORTS
# =============================================================================

import json
import logging
import sys
import warnings
from dataclasses import dataclass, field
from datetime import datetime
from pathlib import Path
from typing import Any, Dict, List, Optional, Tuple

import numpy as np
import pandas as pd
from scipy import stats
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, calinski_harabasz_score

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns

warnings.filterwarnings('ignore')

In [4]:
# =============================================================================
# CONFIGURACION DE VISUALIZACION
# =============================================================================

plt.rcParams.update({
    'figure.figsize': (12, 8),
    'figure.dpi': 150,
    'savefig.dpi': 300,
    'font.size': 10,
    'axes.titlesize': 12,
    'axes.labelsize': 10,
    'xtick.labelsize': 9,
    'ytick.labelsize': 9,
    'legend.fontsize': 9,
    'axes.grid': True,
    'grid.alpha': 0.3,
    'axes.spines.top': False,
    'axes.spines.right': False
})

sns.set_style("whitegrid")

# Paleta de colores para modelos
COLORES_MODELOS = {
    'yolov8': '#2ecc71',
    'oneformer': '#3498db',
    'sam2': '#e74c3c',
    'sam2_prompts': '#9b59b6',
    'mask2former': '#f39c12',
    'bodypix': '#1abc9c'
}

# Colores para clusters de dificultad
COLORES_CLUSTERS = {0: '#27ae60', 1: '#f1c40f', 2: '#e74c3c'}
NOMBRES_CLUSTERS = {0: 'Facil', 1: 'Medio', 2: 'Dificil'}

# Colores para categorias de metricas
COLORES_CATEGORIAS = {
    'clasicas': '#3498db',
    'distancias_borde': '#e67e22',
    'geometricas': '#2ecc71',
    'haralick_interior': '#e74c3c',
    'haralick_borde': '#9b59b6',
    'intensidad': '#f39c12'
}

In [5]:
# =============================================================================
# CONFIGURACION
# =============================================================================

@dataclass
class ConfiguracionFase2C:
    """Configuracion centralizada para Fase 2C."""
    ruta_base_tfm: Path
    ruta_datos_fase2b: Path = None
    ruta_salida: Path = None
    n_componentes_pca: int = 5
    n_clusters: int = 3
    umbral_redundancia: float = 0.90
    umbral_correlacion_significativa: float = 0.30
    random_state: int = 42

    def __post_init__(self):
        if self.ruta_datos_fase2b is None:
            self.ruta_datos_fase2b = (
                self.ruta_base_tfm / "3_Analisis" / "fase2b_correlaciones" /
                "metricas_fusionadas.csv"
            )
        if self.ruta_salida is None:
            self.ruta_salida = self.ruta_base_tfm / "3_Analisis" / "fase2c_pca_clustering"

In [6]:
# =============================================================================
# DEFINICION DE CATEGORIAS DE METRICAS
# =============================================================================

# NOTA: boundary_iou EXCLUIDO por valores incorrectos (implementacion Canny sin tolerancia)
# Se usan chamfer_distance y hausdorff_distance como metricas de calidad de borde

METRICAS_POR_CATEGORIA = {
    'identificadores': [
        'codigo_foto', 'config_codigo', 'modelo'
    ],

    # Metricas clasicas de segmentacion (SIN boundary_iou)
    'clasicas': [
        'iou', 'dice', 'precision', 'recall', 'f1_score'
    ],

    # Metricas de calidad de borde (REEMPLAZAN boundary_iou)
    'distancias_borde': [
        'chamfer_distance',      # Error promedio del borde (menor = mejor)
        'hausdorff_distance'     # Error maximo del borde (menor = mejor)
    ],

    # Metricas geometricas Shapely
    'geometricas': [
        'area_px', 'perimetro_px', 'centroide_x', 'centroide_y',
        'bbox_xmin', 'bbox_ymin', 'bbox_xmax', 'bbox_ymax',
        'aspect_ratio', 'orientacion_grados', 'solidity', 'compacidad',
        'circularity', 'elongation', 'rectangularity',
        'num_componentes', 'ratio_componente_principal',
        'distancia_centro_px', 'zona_tercios', 'recorte_bordes_porcentaje',
        'espacio_negativo',
        'desplazamiento_centroide_px', 'diferencia_orientacion_grados',
        'ratio_solidity', 'diferencia_compacidad', 'symmetric_difference_area_px'
    ],

    # Haralick interior de la mascara (13 features)
    'haralick_interior': [
        'haralick_interior_angular_second_moment',
        'haralick_interior_contrast',
        'haralick_interior_correlation',
        'haralick_interior_sum_of_squares_variance',
        'haralick_interior_inverse_difference_moment',
        'haralick_interior_sum_average',
        'haralick_interior_sum_variance',
        'haralick_interior_sum_entropy',
        'haralick_interior_entropy',
        'haralick_interior_difference_variance',
        'haralick_interior_difference_entropy',
        'haralick_interior_information_measure_correlation_1',
        'haralick_interior_information_measure_correlation_2'
    ],

    # Haralick borde de la mascara (13 features)
    'haralick_borde': [
        'haralick_borde_angular_second_moment',
        'haralick_borde_contrast',
        'haralick_borde_correlation',
        'haralick_borde_sum_of_squares_variance',
        'haralick_borde_inverse_difference_moment',
        'haralick_borde_sum_average',
        'haralick_borde_sum_variance',
        'haralick_borde_sum_entropy',
        'haralick_borde_entropy',
        'haralick_borde_difference_variance',
        'haralick_borde_difference_entropy',
        'haralick_borde_information_measure_correlation_1',
        'haralick_borde_information_measure_correlation_2'
    ],

    # Estadisticas de intensidad
    'intensidad': [
        'intensidad_media', 'intensidad_std', 'intensidad_min', 'intensidad_max'
    ]
}

# Metricas seleccionadas para PCA global (subconjunto representativo)
METRICAS_PCA_GLOBAL = [
    # Clasicas
    'iou', 'dice', 'precision', 'recall', 'f1_score',
    # Distancias de borde (reemplazan boundary_iou)
    'chamfer_distance', 'hausdorff_distance',
    # Geometricas clave
    'solidity', 'compacidad', 'circularity', 'elongation', 'aspect_ratio',
    'espacio_negativo', 'desplazamiento_centroide_px',
    # Haralick interior (top 5)
    'haralick_interior_contrast', 'haralick_interior_correlation',
    'haralick_interior_entropy', 'haralick_interior_homogeneity',
    'haralick_interior_energy',
    # Haralick borde (top 5)
    'haralick_borde_contrast', 'haralick_borde_correlation',
    'haralick_borde_entropy', 'haralick_borde_homogeneity',
    'haralick_borde_energy'
]

In [7]:
# =============================================================================
# FUNCIONES DE UTILIDAD
# =============================================================================

def convertir_a_serializable(obj: Any) -> Any:
    """Convierte tipos numpy/pandas a tipos Python nativos para JSON."""
    if isinstance(obj, dict):
        return {k: convertir_a_serializable(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [convertir_a_serializable(v) for v in obj]
    elif isinstance(obj, (np.integer, np.int64, np.int32)):
        return int(obj)
    elif isinstance(obj, (np.floating, np.float64, np.float32)):
        if np.isnan(obj) or np.isinf(obj):
            return None
        return float(obj)
    elif isinstance(obj, (np.bool_, bool)):
        return bool(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    elif pd.isna(obj):
        return None
    return obj

In [8]:
# =============================================================================
# CLASE: CARGADOR DE DATOS
# =============================================================================

class CargadorDatos:
    """Carga y prepara datos para Fase 2C."""

    def __init__(self, config: ConfiguracionFase2C, logger: logging.Logger):
        self.config = config
        self.logger = logger

    def cargar_datos(self) -> pd.DataFrame:
        """Carga el dataset fusionado de Fase 2B."""
        self.logger.info(f"Cargando datos de: {self.config.ruta_datos_fase2b}")

        if not self.config.ruta_datos_fase2b.exists():
            raise FileNotFoundError(f"No existe: {self.config.ruta_datos_fase2b}")

        df = pd.read_csv(self.config.ruta_datos_fase2b)

        self.logger.info(f"  Filas: {len(df)}")
        self.logger.info(f"  Columnas: {len(df.columns)}")
        self.logger.info(f"  Modelos: {df['modelo'].nunique()}")
        self.logger.info(f"  Fotografias: {df['codigo_foto'].nunique()}")

        return df

    def identificar_metricas_disponibles(self, df: pd.DataFrame) -> Dict[str, List[str]]:
        """Identifica metricas disponibles por categoria en el dataset."""
        self.logger.info("Identificando metricas disponibles por categoria...")

        metricas_disponibles = {}
        total_disponibles = 0

        for categoria, metricas in METRICAS_POR_CATEGORIA.items():
            if categoria == 'identificadores':
                continue

            disponibles = [m for m in metricas if m in df.columns and df[m].notna().sum() > 0]
            metricas_disponibles[categoria] = disponibles
            total_disponibles += len(disponibles)

            self.logger.info(f"  {categoria}: {len(disponibles)}/{len(metricas)} metricas")

        self.logger.info(f"  TOTAL metricas disponibles: {total_disponibles}")

        return metricas_disponibles

    def preparar_matriz_pca(self, df: pd.DataFrame,
                            metricas: List[str]) -> Tuple[np.ndarray, pd.DataFrame, List[str]]:
        """
        Prepara matriz estandarizada para PCA.

        Args:
            df: DataFrame con datos
            metricas: Lista de metricas a incluir

        Returns:
            Tupla (matriz_estandarizada, metadata, nombres_features)
        """
        # Filtrar metricas disponibles
        metricas_validas = [m for m in metricas if m in df.columns]

        if len(metricas_validas) < 3:
            raise ValueError(f"Insuficientes metricas validas: {len(metricas_validas)}")

        # Extraer metricas
        df_metricas = df[metricas_validas].copy()

        # Metadata
        cols_meta = ['modelo', 'config_codigo', 'codigo_foto']
        cols_meta = [c for c in cols_meta if c in df.columns]
        df_meta = df[cols_meta].copy()

        # Eliminar filas con demasiados NaN (>50%)
        umbral_nan = len(metricas_validas) * 0.5
        mask_valido = df_metricas.notna().sum(axis=1) >= umbral_nan

        df_metricas = df_metricas[mask_valido].reset_index(drop=True)
        df_meta = df_meta[mask_valido].reset_index(drop=True)

        self.logger.info(f"  Filas validas: {len(df_metricas)}/{len(df)}")

        # Imputar NaN restantes con mediana
        for col in df_metricas.columns:
            if df_metricas[col].isna().any():
                mediana = df_metricas[col].median()
                df_metricas[col].fillna(mediana, inplace=True)

        # Estandarizar
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(df_metricas)

        return X_scaled, df_meta, metricas_validas

In [9]:
# =============================================================================
# CLASE: ANALIZADOR PCA
# =============================================================================

class AnalizadorPCA:
    """Ejecuta analisis PCA sobre metricas de segmentacion."""

    def __init__(self, config: ConfiguracionFase2C, logger: logging.Logger):
        self.config = config
        self.logger = logger

    def ejecutar_pca(self, X: np.ndarray, nombres_features: List[str],
                     n_componentes: int = None) -> Dict[str, Any]:
        """
        Ejecuta PCA.

        Args:
            X: Matriz estandarizada (n_samples, n_features)
            nombres_features: Nombres de las features
            n_componentes: Numero de componentes (default: config)

        Returns:
            Diccionario con resultados PCA
        """
        if n_componentes is None:
            n_componentes = min(
                self.config.n_componentes_pca,
                X.shape[1],
                X.shape[0] - 1
            )

        self.logger.info(f"  Ejecutando PCA con {n_componentes} componentes...")

        pca = PCA(n_components=n_componentes, random_state=self.config.random_state)
        scores = pca.fit_transform(X)

        var_explicada = pca.explained_variance_ratio_
        var_acumulada = np.cumsum(var_explicada)

        self.logger.info(f"  Varianza explicada por PC1: {var_explicada[0]*100:.1f}%")
        self.logger.info(f"  Varianza total ({n_componentes} PCs): {var_acumulada[-1]*100:.1f}%")

        return {
            'pca_model': pca,
            'scores': scores,
            'loadings': pca.components_,
            'varianza_explicada': var_explicada,
            'varianza_acumulada': var_acumulada,
            'n_componentes': n_componentes,
            'feature_names': nombres_features
        }

    def ejecutar_pca_por_categoria(self, df: pd.DataFrame,
                                    metricas_por_cat: Dict[str, List[str]]) -> Dict[str, Dict]:
        """
        Ejecuta PCA separado por cada categoria de metricas.

        Returns:
            Diccionario {categoria: resultados_pca}
        """
        self.logger.info("Ejecutando PCA por categoria...")

        resultados = {}

        for categoria, metricas in metricas_por_cat.items():
            if len(metricas) < 3:
                self.logger.warning(f"  {categoria}: Insuficientes metricas ({len(metricas)}), omitiendo")
                continue

            # Preparar datos de esta categoria
            df_cat = df[metricas].dropna(thresh=len(metricas)//2)

            if len(df_cat) < 20:
                self.logger.warning(f"  {categoria}: Insuficientes observaciones ({len(df_cat)}), omitiendo")
                continue

            # Imputar NaN
            for col in df_cat.columns:
                if df_cat[col].isna().any():
                    df_cat[col].fillna(df_cat[col].median(), inplace=True)

            # Estandarizar
            scaler = StandardScaler()
            X = scaler.fit_transform(df_cat)

            # PCA
            n_comp = min(5, len(metricas), len(df_cat) - 1)
            resultado = self.ejecutar_pca(X, metricas, n_comp)

            resultados[categoria] = resultado

            self.logger.info(f"  {categoria}: {n_comp} PCs, {resultado['varianza_acumulada'][-1]*100:.1f}% varianza")

        return resultados

    def obtener_top_loadings(self, resultado_pca: Dict, n_top: int = 5) -> Dict[str, List]:
        """Obtiene las variables con mayores loadings absolutos por componente."""
        top_por_pc = {}

        loadings = resultado_pca['loadings']
        nombres = resultado_pca['feature_names']

        for i in range(resultado_pca['n_componentes']):
            # Ordenar por valor absoluto
            indices = np.argsort(np.abs(loadings[i]))[::-1][:n_top]
            top_por_pc[f'PC{i+1}'] = [
                {'feature': nombres[idx], 'loading': float(loadings[i, idx])}
                for idx in indices
            ]

        return top_por_pc

In [10]:
# =============================================================================
# CLASE: ANALIZADOR CLUSTERING
# =============================================================================

class AnalizadorClustering:
    """Clustering de fotografias por nivel de dificultad."""

    def __init__(self, config: ConfiguracionFase2C, logger: logging.Logger):
        self.config = config
        self.logger = logger
        self.labels = None
        self.df_features = None

    def preparar_features_fotografias(self, df: pd.DataFrame) -> pd.DataFrame:
        """
        Prepara features agregadas por fotografia para clustering.

        Cada fotografia se caracteriza por:
        - IoU medio, std, min, max
        - Dice medio
        - Chamfer distance medio (calidad de borde)
        - Numero de modelos con IoU > 0.8
        """
        self.logger.info("Preparando features de fotografias para clustering...")

        # Agregar por foto
        agg_dict = {'iou': ['mean', 'std', 'min', 'max']}

        if 'dice' in df.columns:
            agg_dict['dice'] = 'mean'

        if 'chamfer_distance' in df.columns:
            agg_dict['chamfer_distance'] = 'mean'

        if 'hausdorff_distance' in df.columns:
            agg_dict['hausdorff_distance'] = 'mean'

        features = df.groupby('codigo_foto').agg(agg_dict)
        features.columns = ['_'.join(col).strip('_') for col in features.columns]
        features = features.reset_index()

        # Renombrar para claridad
        rename_map = {
            'iou_mean': 'iou_mean',
            'iou_std': 'iou_std',
            'iou_min': 'iou_min',
            'iou_max': 'iou_max'
        }
        features = features.rename(columns=rename_map)

        # Rango IoU (variabilidad entre modelos)
        features['iou_rango'] = features['iou_max'] - features['iou_min']

        # Contar modelos con IoU > 0.8 (modelos que funcionan bien)
        modelos_buenos = df[df['iou'] > 0.8].groupby('codigo_foto')['modelo'].nunique()
        modelos_buenos = modelos_buenos.reset_index()
        modelos_buenos.columns = ['codigo_foto', 'n_modelos_buenos']

        features = features.merge(modelos_buenos, on='codigo_foto', how='left')
        features['n_modelos_buenos'].fillna(0, inplace=True)

        self.logger.info(f"  Fotografias: {len(features)}")
        self.logger.info(f"  Features: {list(features.columns)}")

        self.df_features = features
        return features

    def ejecutar_kmeans(self, df_features: pd.DataFrame = None) -> Dict:
        """
        Ejecuta K-means clustering.

        Clusters interpretados como:
        - Cluster 0: Facil (alto IoU medio, baja varianza)
        - Cluster 1: Medio
        - Cluster 2: Dificil (bajo IoU medio, alta varianza)
        """
        if df_features is None:
            df_features = self.df_features

        self.logger.info(f"Ejecutando K-means (k={self.config.n_clusters})...")

        # Features para clustering
        cols_clustering = ['iou_mean', 'iou_std', 'iou_rango', 'n_modelos_buenos']
        cols_disponibles = [c for c in cols_clustering if c in df_features.columns]

        X = df_features[cols_disponibles].fillna(0).values

        # Estandarizar
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)

        # K-means
        kmeans = KMeans(
            n_clusters=self.config.n_clusters,
            random_state=self.config.random_state,
            n_init=10
        )
        labels_orig = kmeans.fit_predict(X_scaled)

        # Metricas de calidad
        silhouette = silhouette_score(X_scaled, labels_orig)
        calinski = calinski_harabasz_score(X_scaled, labels_orig)

        self.logger.info(f"  Silhouette Score: {silhouette:.3f}")
        self.logger.info(f"  Calinski-Harabasz: {calinski:.1f}")

        # Reordenar clusters: 0=facil (mayor IoU), 2=dificil (menor IoU)
        df_temp = df_features.copy()
        df_temp['cluster_orig'] = labels_orig
        medias_iou = df_temp.groupby('cluster_orig')['iou_mean'].mean()
        orden = medias_iou.sort_values(ascending=False).index.tolist()
        mapeo = {old: new for new, old in enumerate(orden)}
        self.labels = np.array([mapeo[l] for l in labels_orig])

        # Caracterizacion de clusters
        df_temp['cluster'] = self.labels
        caracterizacion = {}

        for c in range(self.config.n_clusters):
            df_c = df_temp[df_temp['cluster'] == c]
            caracterizacion[c] = {
                'nombre': NOMBRES_CLUSTERS[c],
                'n_fotos': int(len(df_c)),
                'iou_mean': float(df_c['iou_mean'].mean()),
                'iou_std_mean': float(df_c['iou_std'].mean()),
                'iou_rango_mean': float(df_c['iou_rango'].mean()),
                'fotos': df_c['codigo_foto'].tolist()
            }
            self.logger.info(
                f"  Cluster {c} ({NOMBRES_CLUSTERS[c]}): "
                f"{len(df_c)} fotos, IoU medio={df_c['iou_mean'].mean():.3f}"
            )

        return {
            'silhouette': float(silhouette),
            'calinski_harabasz': float(calinski),
            'n_clusters': self.config.n_clusters,
            'caracterizacion': caracterizacion
        }

    def ejecutar_jerarquico(self, df_features: pd.DataFrame = None) -> np.ndarray:
        """Ejecuta clustering jerarquico (para dendrograma)."""
        if df_features is None:
            df_features = self.df_features

        cols = ['iou_mean', 'iou_std', 'iou_rango']
        cols_disponibles = [c for c in cols if c in df_features.columns]

        X = df_features[cols_disponibles].fillna(0).values

        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)

        return linkage(X_scaled, method='ward')

In [11]:
# =============================================================================
# CLASE: ANALIZADOR CORRELACIONES
# =============================================================================

class AnalizadorCorrelaciones:
    """Analiza correlaciones entre metricas de segmentacion."""

    def __init__(self, config: ConfiguracionFase2C, logger: logging.Logger):
        self.config = config
        self.logger = logger

    def calcular_matriz_correlaciones(self, df: pd.DataFrame,
                                       metricas: List[str]) -> pd.DataFrame:
        """Calcula matriz de correlacion Pearson entre metricas."""
        self.logger.info(f"Calculando matriz de correlaciones ({len(metricas)} metricas)...")

        metricas_validas = [m for m in metricas if m in df.columns]
        df_metricas = df[metricas_validas].dropna(thresh=len(metricas_validas)//2)

        matriz = df_metricas.corr(method='pearson')

        self.logger.info(f"  Matriz: {matriz.shape[0]}x{matriz.shape[1]}")

        return matriz

    def identificar_metricas_redundantes(self, matriz: pd.DataFrame) -> pd.DataFrame:
        """Identifica pares de metricas altamente correlacionadas (redundantes)."""
        self.logger.info(f"Identificando metricas redundantes (|r| > {self.config.umbral_redundancia})...")

        redundantes = []
        n = len(matriz)

        for i in range(n):
            for j in range(i + 1, n):
                r = matriz.iloc[i, j]
                if abs(r) > self.config.umbral_redundancia:
                    redundantes.append({
                        'metrica_1': matriz.index[i],
                        'metrica_2': matriz.columns[j],
                        'correlacion': float(r),
                        'tipo': 'positiva' if r > 0 else 'negativa'
                    })

        df_red = pd.DataFrame(redundantes)

        if len(df_red) > 0:
            df_red = df_red.sort_values('correlacion', key=abs, ascending=False)

        self.logger.info(f"  Pares redundantes encontrados: {len(df_red)}")

        return df_red

    def calcular_correlaciones_con_iou(self, df: pd.DataFrame,
                                        metricas: List[str]) -> pd.DataFrame:
        """Calcula correlaciones de todas las metricas con IoU."""
        self.logger.info("Calculando correlaciones de metricas con IoU...")

        correlaciones = []

        for metrica in metricas:
            if metrica == 'iou' or metrica not in df.columns:
                continue

            mask = df[metrica].notna() & df['iou'].notna()
            n = mask.sum()

            if n < 20:
                continue

            try:
                r, p = stats.pearsonr(df.loc[mask, metrica], df.loc[mask, 'iou'])

                correlaciones.append({
                    'metrica': metrica,
                    'correlacion_iou': float(r),
                    'p_valor': float(p),
                    'n_observaciones': int(n),
                    'significativo': p < 0.05,
                    'magnitud': 'alta' if abs(r) > 0.5 else 'media' if abs(r) > 0.3 else 'baja'
                })
            except Exception:
                pass

        df_corr = pd.DataFrame(correlaciones)

        if len(df_corr) > 0:
            df_corr = df_corr.sort_values('correlacion_iou', key=abs, ascending=False)

        n_sig = df_corr['significativo'].sum() if len(df_corr) > 0 else 0
        self.logger.info(f"  Correlaciones calculadas: {len(df_corr)}")
        self.logger.info(f"  Significativas (p<0.05): {n_sig}")

        return df_corr

In [12]:
# =============================================================================
# CLASE: GENERADOR VISUALIZACIONES
# =============================================================================

class GeneradorVisualizaciones:
    """Genera visualizaciones para Fase 2C."""

    def __init__(self, config: ConfiguracionFase2C, logger: logging.Logger):
        self.config = config
        self.logger = logger
        self.ruta_viz = config.ruta_salida / "visualizaciones"

    def _crear_directorio(self):
        """Crea directorio de visualizaciones si no existe."""
        self.ruta_viz.mkdir(parents=True, exist_ok=True)

    def generar_scree_plot(self, resultado_pca: Dict, titulo: str = "PCA") -> Path:
        """Genera scree plot de varianza explicada."""
        self._crear_directorio()

        fig, ax = plt.subplots(figsize=(10, 6))

        n = resultado_pca['n_componentes']
        x = range(1, n + 1)
        var = resultado_pca['varianza_explicada'] * 100
        acum = resultado_pca['varianza_acumulada'] * 100

        # Barras de varianza individual
        bars = ax.bar(x, var, color='steelblue', alpha=0.7, label='Individual')

        # Linea de varianza acumulada
        ax.plot(x, acum, 'ro-', linewidth=2, markersize=8, label='Acumulada')

        # Linea de referencia 80%
        ax.axhline(y=80, color='gray', linestyle='--', alpha=0.5, label='80%')

        # Etiquetas en barras
        for i, v in enumerate(var):
            ax.text(i + 1, v + 1.5, f'{v:.1f}%', ha='center', fontsize=9)

        ax.set_xlabel('Componente Principal')
        ax.set_ylabel('Varianza Explicada (%)')
        ax.set_title(f'{titulo}: Varianza Explicada por Componente', fontweight='bold')
        ax.set_xticks(x)
        ax.set_xticklabels([f'PC{i}' for i in x])
        ax.legend(loc='right')
        ax.set_ylim(0, 105)

        plt.tight_layout()

        nombre_archivo = titulo.lower().replace(' ', '_').replace(':', '').replace('/', '_')
        ruta = self.ruta_viz / f'scree_{nombre_archivo}.png'
        fig.savefig(ruta, dpi=300, bbox_inches='tight')
        plt.close(fig)

        self.logger.info(f"  Guardado: {ruta.name}")
        return ruta

    def generar_biplot(self, resultado_pca: Dict, df_meta: pd.DataFrame,
                       titulo: str = "PCA") -> Path:
        """Genera biplot con scores y loadings."""
        self._crear_directorio()

        fig, ax = plt.subplots(figsize=(12, 10))

        scores = resultado_pca['scores'][:, :2]
        loadings = resultado_pca['loadings'][:2, :].T
        nombres = resultado_pca['feature_names']

        # Scatter de scores por modelo
        if 'modelo' in df_meta.columns:
            modelos = df_meta['modelo'].values[:len(scores)]
            for modelo in np.unique(modelos):
                mask = modelos == modelo
                color = COLORES_MODELOS.get(modelo, '#666666')
                ax.scatter(scores[mask, 0], scores[mask, 1],
                          c=color, label=modelo, alpha=0.5, s=30, edgecolors='white')
        else:
            ax.scatter(scores[:, 0], scores[:, 1], alpha=0.5, s=30)

        # Flechas de loadings
        scale = np.abs(scores).max() / np.abs(loadings).max() * 0.8

        for i, (name, loading) in enumerate(zip(nombres, loadings)):
            ax.arrow(0, 0, loading[0] * scale, loading[1] * scale,
                    head_width=0.08, head_length=0.06, fc='darkred', ec='darkred', alpha=0.7)

            # Nombre abreviado
            nombre_corto = name[:20] + '...' if len(name) > 20 else name
            offset = 1.12 if loading[1] >= 0 else 1.08
            ax.text(loading[0] * scale * offset, loading[1] * scale * offset,
                   nombre_corto, fontsize=7, ha='center', color='darkred')

        ax.axhline(y=0, color='gray', linewidth=0.5)
        ax.axvline(x=0, color='gray', linewidth=0.5)

        var = resultado_pca['varianza_explicada'] * 100
        ax.set_xlabel(f'PC1 ({var[0]:.1f}%)')
        ax.set_ylabel(f'PC2 ({var[1]:.1f}%)')
        ax.set_title(f'{titulo}: Biplot', fontweight='bold')

        if 'modelo' in df_meta.columns:
            ax.legend(loc='upper right', title='Modelo', fontsize=8)

        plt.tight_layout()

        nombre_archivo = titulo.lower().replace(' ', '_').replace(':', '').replace('/', '_')
        ruta = self.ruta_viz / f'biplot_{nombre_archivo}.png'
        fig.savefig(ruta, dpi=300, bbox_inches='tight')
        plt.close(fig)

        self.logger.info(f"  Guardado: {ruta.name}")
        return ruta

    def generar_heatmap_correlaciones(self, matriz: pd.DataFrame,
                                       titulo: str = "Correlaciones") -> Path:
        """Genera heatmap de matriz de correlaciones."""
        self._crear_directorio()

        n = len(matriz)
        size = max(10, min(20, n * 0.5))

        fig, ax = plt.subplots(figsize=(size, size * 0.9))

        # Mascara triangular superior
        mask = np.triu(np.ones_like(matriz, dtype=bool), k=1)

        sns.heatmap(
            matriz,
            mask=mask,
            annot=n <= 15,
            fmt='.2f' if n <= 15 else '',
            cmap='RdBu_r',
            center=0,
            vmin=-1, vmax=1,
            square=True,
            linewidths=0.5 if n <= 20 else 0,
            cbar_kws={'shrink': 0.8, 'label': 'Correlacion'},
            ax=ax,
            annot_kws={'size': 7}
        )

        ax.set_title(f'{titulo}', fontweight='bold', pad=20)

        # Rotar etiquetas
        plt.xticks(rotation=45, ha='right')
        plt.yticks(rotation=0)

        plt.tight_layout()

        nombre_archivo = titulo.lower().replace(' ', '_').replace(':', '').replace('/', '_')
        ruta = self.ruta_viz / f'heatmap_{nombre_archivo}.png'
        fig.savefig(ruta, dpi=300, bbox_inches='tight')
        plt.close(fig)

        self.logger.info(f"  Guardado: {ruta.name}")
        return ruta

    def generar_dendrograma(self, linkage_matrix: np.ndarray,
                            etiquetas: List[str]) -> Path:
        """Genera dendrograma de clustering jerarquico."""
        self._crear_directorio()

        fig, ax = plt.subplots(figsize=(14, 8))

        # Abreviar etiquetas
        etiquetas_cortas = [e.replace('_DSC', '').replace('_', '') for e in etiquetas]

        dendrogram(
            linkage_matrix,
            labels=etiquetas_cortas,
            leaf_rotation=90,
            leaf_font_size=9,
            ax=ax,
            color_threshold=0.7 * max(linkage_matrix[:, 2])
        )

        ax.set_xlabel('Fotografia')
        ax.set_ylabel('Distancia (Ward)')
        ax.set_title('Clustering Jerarquico de Fotografias por Dificultad', fontweight='bold')

        plt.tight_layout()

        ruta = self.ruta_viz / 'dendrograma_fotografias.png'
        fig.savefig(ruta, dpi=300, bbox_inches='tight')
        plt.close(fig)

        self.logger.info(f"  Guardado: {ruta.name}")
        return ruta

    def generar_scatter_clusters(self, df_features: pd.DataFrame,
                                  labels: np.ndarray) -> Path:
        """Genera scatter plot de clusters."""
        self._crear_directorio()

        fig, axes = plt.subplots(1, 2, figsize=(14, 6))

        df_plot = df_features.copy()
        df_plot['cluster'] = labels

        # Panel 1: IoU mean vs std
        ax = axes[0]
        for c in range(3):
            mask = df_plot['cluster'] == c
            ax.scatter(
                df_plot.loc[mask, 'iou_mean'],
                df_plot.loc[mask, 'iou_std'],
                c=COLORES_CLUSTERS[c],
                label=NOMBRES_CLUSTERS[c],
                s=100, alpha=0.7, edgecolors='white', linewidth=1
            )

            # Etiquetas de fotos
            for _, row in df_plot[mask].iterrows():
                ax.annotate(row['codigo_foto'].replace('_DSC', ''),
                           (row['iou_mean'], row['iou_std']),
                           fontsize=6, alpha=0.7)

        ax.set_xlabel('IoU Medio (todos los modelos)')
        ax.set_ylabel('IoU Desviacion Estandar')
        ax.set_title('Clusters: Rendimiento vs Variabilidad')
        ax.legend(title='Dificultad')

        # Panel 2: IoU mean vs rango
        ax = axes[1]
        for c in range(3):
            mask = df_plot['cluster'] == c
            ax.scatter(
                df_plot.loc[mask, 'iou_mean'],
                df_plot.loc[mask, 'iou_rango'],
                c=COLORES_CLUSTERS[c],
                label=NOMBRES_CLUSTERS[c],
                s=100, alpha=0.7, edgecolors='white', linewidth=1
            )

        ax.set_xlabel('IoU Medio')
        ax.set_ylabel('Rango IoU (max - min)')
        ax.set_title('Clusters: Rendimiento vs Rango entre Modelos')
        ax.legend(title='Dificultad')

        plt.tight_layout()

        ruta = self.ruta_viz / 'clusters_fotografias.png'
        fig.savefig(ruta, dpi=300, bbox_inches='tight')
        plt.close(fig)

        self.logger.info(f"  Guardado: {ruta.name}")
        return ruta

    def generar_barplot_correlaciones_iou(self, df_corr: pd.DataFrame,
                                           n_top: int = 25) -> Path:
        """Genera barplot horizontal de correlaciones con IoU."""
        self._crear_directorio()

        df_top = df_corr.head(n_top).copy()

        fig, ax = plt.subplots(figsize=(10, 10))

        # Colores segun signo
        colores = ['#e74c3c' if r < 0 else '#27ae60'
                   for r in df_top['correlacion_iou']]

        y_pos = range(len(df_top))
        bars = ax.barh(y_pos, df_top['correlacion_iou'], color=colores, alpha=0.7)

        ax.set_yticks(y_pos)
        ax.set_yticklabels(df_top['metrica'], fontsize=8)
        ax.set_xlabel('Correlacion con IoU')
        ax.set_title(f'Top {n_top} Metricas mas Correlacionadas con IoU', fontweight='bold')
        ax.axvline(x=0, color='black', linewidth=0.5)
        ax.set_xlim(-1, 1)
        ax.invert_yaxis()

        # Leyenda
        legend_elements = [
            mpatches.Patch(color='#27ae60', label='Correlacion positiva'),
            mpatches.Patch(color='#e74c3c', label='Correlacion negativa')
        ]
        ax.legend(handles=legend_elements, loc='lower right')

        plt.tight_layout()

        ruta = self.ruta_viz / 'correlaciones_con_iou.png'
        fig.savefig(ruta, dpi=300, bbox_inches='tight')
        plt.close(fig)

        self.logger.info(f"  Guardado: {ruta.name}")
        return ruta

In [13]:
# =============================================================================
# CLASE: ORQUESTADOR FASE 2C
# =============================================================================

class OrquestadorFase2C:
    """Orquesta la ejecucion completa de Fase 2C."""

    def __init__(self, config: ConfiguracionFase2C):
        self.config = config
        self.logger = self._configurar_logger()

        # Componentes
        self.cargador = CargadorDatos(config, self.logger)
        self.analizador_pca = AnalizadorPCA(config, self.logger)
        self.analizador_clustering = AnalizadorClustering(config, self.logger)
        self.analizador_corr = AnalizadorCorrelaciones(config, self.logger)
        self.generador_viz = GeneradorVisualizaciones(config, self.logger)

        # Resultados
        self.resultados = {}
        self.df = None

    def _configurar_logger(self) -> logging.Logger:
        """Configura logger para Fase 2C."""
        logger = logging.getLogger('Fase2C')
        logger.setLevel(logging.INFO)
        logger.handlers = []

        handler = logging.StreamHandler(sys.stdout)
        handler.setLevel(logging.INFO)
        formatter = logging.Formatter(
            '[%(asctime)s] %(levelname)-8s | %(message)s',
            datefmt='%H:%M:%S'
        )
        handler.setFormatter(formatter)
        logger.addHandler(handler)

        return logger

    def ejecutar(self) -> Dict:
        """Ejecuta el pipeline completo de Fase 2C."""
        self.logger.info("=" * 70)
        self.logger.info("FASE 2C - PCA, CLUSTERING Y ANALISIS DE REDUNDANCIA")
        self.logger.info("Version 3.0 - Usando Chamfer/Hausdorff en lugar de Boundary IoU")
        self.logger.info("=" * 70)

        inicio = datetime.now()

        # Paso 1: Cargar datos
        self.logger.info("\n[PASO 1/8] Cargando datos de Fase 2B...")
        self.df = self.cargador.cargar_datos()

        # Paso 2: Identificar metricas
        self.logger.info("\n[PASO 2/8] Identificando metricas disponibles...")
        metricas_por_cat = self.cargador.identificar_metricas_disponibles(self.df)
        self.resultados['metricas_por_categoria'] = {k: len(v) for k, v in metricas_por_cat.items()}

        # Paso 3: PCA global
        self.logger.info("\n[PASO 3/8] Ejecutando PCA global...")
        metricas_pca = [m for m in METRICAS_PCA_GLOBAL if m in self.df.columns]
        self.logger.info(f"  Metricas para PCA global: {len(metricas_pca)}")

        X_pca, df_meta, nombres_features = self.cargador.preparar_matriz_pca(self.df, metricas_pca)
        self.resultados['pca_global'] = self.analizador_pca.ejecutar_pca(X_pca, nombres_features)
        self.resultados['top_loadings'] = self.analizador_pca.obtener_top_loadings(
            self.resultados['pca_global']
        )

        # Paso 4: PCA por categoria
        self.logger.info("\n[PASO 4/8] Ejecutando PCA por categoria...")
        self.resultados['pca_categorias'] = self.analizador_pca.ejecutar_pca_por_categoria(
            self.df, metricas_por_cat
        )

        # Paso 5: Clustering
        self.logger.info("\n[PASO 5/8] Ejecutando clustering de fotografias...")
        df_features = self.analizador_clustering.preparar_features_fotografias(self.df)
        self.resultados['clustering'] = self.analizador_clustering.ejecutar_kmeans(df_features)
        linkage_mat = self.analizador_clustering.ejecutar_jerarquico(df_features)

        # Paso 6: Correlaciones
        self.logger.info("\n[PASO 6/8] Analizando correlaciones entre metricas...")
        todas_metricas = []
        for cat, mets in metricas_por_cat.items():
            todas_metricas.extend(mets)

        matriz_corr = self.analizador_corr.calcular_matriz_correlaciones(self.df, todas_metricas)
        self.resultados['redundantes'] = self.analizador_corr.identificar_metricas_redundantes(matriz_corr)
        self.resultados['corr_iou'] = self.analizador_corr.calcular_correlaciones_con_iou(
            self.df, todas_metricas
        )

        # Paso 7: Visualizaciones
        self.logger.info("\n[PASO 7/8] Generando visualizaciones...")

        self.generador_viz.generar_scree_plot(self.resultados['pca_global'], "PCA Global")
        self.generador_viz.generar_biplot(self.resultados['pca_global'], df_meta, "PCA Global")

        # Heatmap (solo si matriz no es muy grande)
        if len(matriz_corr) <= 40:
            self.generador_viz.generar_heatmap_correlaciones(matriz_corr, "Metricas Segmentacion")

        self.generador_viz.generar_dendrograma(linkage_mat, df_features['codigo_foto'].tolist())
        self.generador_viz.generar_scatter_clusters(df_features, self.analizador_clustering.labels)
        self.generador_viz.generar_barplot_correlaciones_iou(self.resultados['corr_iou'])

        # Paso 8: Guardar resultados
        self.logger.info("\n[PASO 8/8] Guardando resultados...")
        self._guardar_resultados(df_features, matriz_corr, df_meta)

        # Resumen final
        duracion = (datetime.now() - inicio).total_seconds()

        self.logger.info("\n" + "=" * 70)
        self.logger.info("FASE 2C COMPLETADA")
        self.logger.info("=" * 70)
        self.logger.info(f"Duracion: {duracion:.1f} segundos")
        self.logger.info(f"Resultados guardados en: {self.config.ruta_salida}")

        return self.resultados

    def _guardar_resultados(self, df_features: pd.DataFrame,
                            matriz_corr: pd.DataFrame,
                            df_meta: pd.DataFrame) -> None:
        """Guarda todos los resultados en archivos."""
        self.config.ruta_salida.mkdir(parents=True, exist_ok=True)

        # --- PCA Global ---
        pca_dir = self.config.ruta_salida / "pca_global"
        pca_dir.mkdir(exist_ok=True)

        pca = self.resultados['pca_global']
        n_comp = pca['n_componentes']
        cols_pc = [f'PC{i+1}' for i in range(n_comp)]

        # Scores
        df_scores = pd.DataFrame(pca['scores'], columns=cols_pc)
        for col in ['modelo', 'config_codigo', 'codigo_foto']:
            if col in df_meta.columns:
                df_scores[col] = df_meta[col].values[:len(df_scores)]
        df_scores.to_csv(pca_dir / 'pca_scores.csv', index=False)

        # Loadings
        df_loadings = pd.DataFrame(
            pca['loadings'].T,
            index=pca['feature_names'],
            columns=cols_pc
        )
        df_loadings.to_csv(pca_dir / 'pca_loadings.csv')

        # Varianza
        df_var = pd.DataFrame({
            'componente': cols_pc,
            'varianza_explicada': pca['varianza_explicada'],
            'varianza_acumulada': pca['varianza_acumulada']
        })
        df_var.to_csv(pca_dir / 'pca_varianza.csv', index=False)

        self.logger.info(f"  PCA global guardado en: {pca_dir}")

        # --- Clustering ---
        cl_dir = self.config.ruta_salida / "clustering"
        cl_dir.mkdir(exist_ok=True)

        df_clusters = df_features[['codigo_foto']].copy()
        df_clusters['cluster'] = self.analizador_clustering.labels
        df_clusters['nombre_cluster'] = df_clusters['cluster'].map(NOMBRES_CLUSTERS)
        df_clusters['iou_mean'] = df_features['iou_mean']
        df_clusters['iou_std'] = df_features['iou_std']
        df_clusters.to_csv(cl_dir / 'clusters_fotografias.csv', index=False)

        # Caracterizacion JSON
        with open(cl_dir / 'caracterizacion_clusters.json', 'w', encoding='utf-8') as f:
            json.dump(
                convertir_a_serializable(self.resultados['clustering']['caracterizacion']),
                f, indent=2, ensure_ascii=False
            )

        self.logger.info(f"  Clustering guardado en: {cl_dir}")

        # --- Correlaciones ---
        corr_dir = self.config.ruta_salida / "correlaciones"
        corr_dir.mkdir(exist_ok=True)

        matriz_corr.to_csv(corr_dir / 'matriz_correlaciones.csv')
        self.resultados['redundantes'].to_csv(corr_dir / 'metricas_redundantes.csv', index=False)
        self.resultados['corr_iou'].to_csv(corr_dir / 'correlaciones_con_iou.csv', index=False)

        self.logger.info(f"  Correlaciones guardadas en: {corr_dir}")

        # --- Resumen JSON ---
        resumen = {
            'timestamp': datetime.now().isoformat(),
            'version': '3.0',
            'nota': 'boundary_iou excluido, usando chamfer/hausdorff',
            'datos': {
                'filas': len(self.df),
                'columnas': len(self.df.columns),
                'modelos': int(self.df['modelo'].nunique()),
                'fotografias': int(self.df['codigo_foto'].nunique())
            },
            'pca_global': {
                'n_componentes': pca['n_componentes'],
                'varianza_total': float(pca['varianza_acumulada'][-1]),
                'varianza_pc1': float(pca['varianza_explicada'][0])
            },
            'clustering': {
                'algoritmo': 'kmeans',
                'n_clusters': self.resultados['clustering']['n_clusters'],
                'silhouette': self.resultados['clustering']['silhouette'],
                'calinski_harabasz': self.resultados['clustering']['calinski_harabasz']
            },
            'correlaciones': {
                'total_metricas': len(matriz_corr),
                'pares_redundantes': len(self.resultados['redundantes']),
                'correlaciones_significativas_iou': int(self.resultados['corr_iou']['significativo'].sum())
            }
        }

        with open(self.config.ruta_salida / 'resumen_fase2c.json', 'w', encoding='utf-8') as f:
            json.dump(resumen, f, indent=2, ensure_ascii=False)

        self.logger.info(f"  Resumen guardado: resumen_fase2c.json")

In [14]:
# =============================================================================
# FUNCION PRINCIPAL
# =============================================================================

def ejecutar_fase2c(ruta_base_tfm: str,
                    ruta_datos: str = None) -> OrquestadorFase2C:
    """
    Ejecuta Fase 2C completa.

    Args:
        ruta_base_tfm: Ruta base del proyecto TFM
        ruta_datos: Ruta opcional al CSV fusionado de Fase 2B

    Returns:
        OrquestadorFase2C con resultados
    """
    config = ConfiguracionFase2C(ruta_base_tfm=Path(ruta_base_tfm))

    if ruta_datos:
        config.ruta_datos_fase2b = Path(ruta_datos)

    orquestador = OrquestadorFase2C(config)
    orquestador.ejecutar()

    return orquestador

In [15]:
# =============================================================================
# MAIN
# =============================================================================

if __name__ == "__main__":

    # Montar Google Drive
    from google.colab import drive
    drive.mount('/content/drive')

    # Rutas
    RUTA_BASE = Path("/content/drive/MyDrive/TFM")
    RUTA_DATOS = RUTA_BASE / "3_Analisis" / "fase2b_correlaciones" / "metricas_fusionadas.csv"

    # Verificar existencia
    if not RUTA_DATOS.exists():
        print(f"ERROR: No existe el archivo de Fase 2B")
        print(f"Ruta esperada: {RUTA_DATOS}")
        print("\nEjecute Fase 2B primero.")
    else:
        print(f"Archivo encontrado: {RUTA_DATOS}")
        print(f"Iniciando Fase 2C...\n")

        # Ejecutar
        orquestador = ejecutar_fase2c(str(RUTA_BASE), str(RUTA_DATOS))

        # Mostrar resumen
        print("\n" + "=" * 70)
        print("RESUMEN DE RESULTADOS")
        print("=" * 70)

        if 'clustering' in orquestador.resultados:
            print("\nCLUSTERS DE FOTOGRAFIAS:")
            for c, info in orquestador.resultados['clustering']['caracterizacion'].items():
                print(f"  {info['nombre']}: {info['n_fotos']} fotos, IoU medio = {info['iou_mean']:.4f}")

        if 'corr_iou' in orquestador.resultados:
            df_corr = orquestador.resultados['corr_iou']
            print(f"\nTOP 5 CORRELACIONES CON IoU:")
            for _, row in df_corr.head(5).iterrows():
                print(f"  {row['metrica']}: r = {row['correlacion_iou']:.3f}")

Mounted at /content/drive
Archivo encontrado: /content/drive/MyDrive/TFM/3_Analisis/fase2b_correlaciones/metricas_fusionadas.csv
Iniciando Fase 2C...

[21:17:49] INFO     | ======================================================================


INFO:Fase2C:======================================================================


[21:17:49] INFO     | FASE 2C - PCA, CLUSTERING Y ANALISIS DE REDUNDANCIA


INFO:Fase2C:FASE 2C - PCA, CLUSTERING Y ANALISIS DE REDUNDANCIA


[21:17:49] INFO     | Version 3.0 - Usando Chamfer/Hausdorff en lugar de Boundary IoU


...

[21:17:55] INFO     | Duracion: 6.3 segundos


INFO:Fase2C:Duracion: 6.3 segundos


[21:17:55] INFO     | Resultados guardados en: /content/drive/MyDrive/TFM/3_Analisis/fase2c_pca_clustering


INFO:Fase2C:Resultados guardados en: /content/drive/MyDrive/TFM/3_Analisis/fase2c_pca_clustering



RESUMEN DE RESULTADOS

CLUSTERS DE FOTOGRAFIAS:
  Facil: 3 fotos, IoU medio = 0.8298
  Medio: 9 fotos, IoU medio = 0.7177
  Dificil: 8 fotos, IoU medio = 0.5733

TOP 5 CORRELACIONES CON IoU:
  f1_score: r = 0.984
  dice: r = 0.984
  precision: r = 0.904
  recall: r = 0.876
  chamfer_distance: r = -0.721
